# Getting Wildfire Data
This Notebook contains code to get wildlife data from US government repository. This Jupyter notebook also contains code that performs basic geodetic computations related to the Wildfire course project. 

The complete [Wildfire dataset](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) can be retrieved from a US government repository. 

This notebook has dependencies on [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html), the [geojson](https://pypi.org/project/geojson/) module and on the wildfire user module. Pyproj and geojson can be installed via pip. The wildfire user module should be downloaded from the course website, unzipped, and moved into the folder pointed to by your PYTHONPATH system variable.

### License
A lot of code used in this notebook was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. 

So, this code by extension is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - August 16, 2024

### Preliminaries
First, we load the json data. For this we install a few libraries and load the data.

In [1]:
%pip install pyproj

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.9 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
#
#    IMPORTS
#
#    Import some standard python modules
import os, json, time
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#
#    The 'wildfire' module is a user module. This module is available from the course website. The module
#    includes one object, a Reader, that can be used to read the GeoJSON files associated with the
#    wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
#    contains a small number of wildfires extracted from the main wildfire dataset.
#
from wildfire.Reader import Reader as WFReader
#
#    There is a GeoJSON reader that you might try if you wanted to read the data. It has its own quirks.
#    There will be an example below that reads the sample file "Wildfire_short_sample.json"
#
import geojson
#

In [3]:
#
#    CONSTANTS
#

#
#    The file below contains the wildfire data. The file has been downloaded from the source cited above.
#
EXTRACT_FILENAME = "USGS_Wildland_Fire_Combined_Dataset.json"
#
#    The user module 'wildfire' contains a Reader object and the sample data. This bit of code finds where that is
#    located on your machine and constructs a path so that the sample data can be loaded. This assumes you have set
#    a PYTHONPATH environment variable to point to the location on your machine where you store python user modules.
#
#    NOTE: if you use Anaconda for virtual python environments, Anaconda will adhere to the PYTHONPATH conventions
#    for user modules.
#
MODULENAME = "wildfire"
MODULEPATH = ""
try:
    ppath = os.environ.get('PYTHONPATH')
    if not ppath: raise
    MODULEPATH = os.path.join(ppath,MODULENAME)
except:
    # Likely here because a PYTHONPATH was not set, show a warning message
    print("Looks like you're not using a 'PYTHONPATH' to specify the location of your python user modules.")
    print("You may have to modify the sample code in this notebook to get the documented behaviors.")
    MODULEPATH = ""

if MODULEPATH:
    SAMPLE_DATA_FILENAME = os.path.join(MODULEPATH,EXTRACT_FILENAME)
else:
    SAMPLE_DATA_FILENAME = EXTRACT_FILENAME
#
# print out where we think we're going to find the sample data
print(f"{SAMPLE_DATA_FILENAME=}")




Looks like you're not using a 'PYTHONPATH' to specify the location of your python user modules.
You may have to modify the sample code in this notebook to get the documented behaviors.
SAMPLE_DATA_FILENAME='USGS_Wildland_Fire_Combined_Dataset.json'


Next lets load the wildfire data using the GeoJSON module

Here, we use the GeoJSON module ([documentation](https://pypi.org/project/geojson/), [GitHub repo](https://github.com/jazzband/geojson)) to load the file.


In [4]:
#
#    Open a file, load it with the geojson loader
#
print(f"Attempting to open '{SAMPLE_DATA_FILENAME}'")
geojson_file = open(SAMPLE_DATA_FILENAME,"r")
print(f"Using GeoJSON module to load sample file '{SAMPLE_DATA_FILENAME}'")
gj_data = geojson.load(geojson_file)
geojson_file.close()
#
#    Print the keys from the object
#
gj_keys = list(gj_data.keys())
print("The loaded JSON dictionary has the following keys:")
print(gj_keys)
print()
#
#    For all GeoJSON type things, the most important part of the file are the 'features'.
#    In the case of the wildfire dataset, each feature is a polygon (ring) of points that define the bounary of a fire
#
count = 0
for feature in gj_data['features']:
    count += 1
    #print(json.dumps(feature,indent=4))
    #time.sleep(0.5)   # this slows the output to fix output rate limits for Jupyter

print(f"Found {count} features in the variable 'gj_data' ")
#

Attempting to open 'USGS_Wildland_Fire_Combined_Dataset.json'
Using GeoJSON module to load sample file 'USGS_Wildland_Fire_Combined_Dataset.json'
The loaded JSON dictionary has the following keys:
['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features']

Found 135061 features in the variable 'gj_data' 


In [5]:
len(gj_data['features'])

135061

In [6]:
#
#    Get the first item in the list of features
#
SLOT = 0
gj_feature = gj_data['features'][SLOT]
#
#    Print everyting in this dictionary (i.e., gj_feature) - it's long
#
print(f"The wildfire feature from slot '{SLOT}' of the loaded gj_data['features']")
print(json.dumps(gj_feature, indent=4))


The wildfire feature from slot '0' of the loaded gj_data['features']
{
    "attributes": {
        "OBJECTID": 1,
        "USGS_Assigned_ID": 1,
        "Assigned_Fire_Type": "Wildfire",
        "Fire_Year": 1860,
        "Fire_Polygon_Tier": 1,
        "Fire_Attribute_Tiers": "1 (1)",
        "GIS_Acres": 3940.20708940724,
        "GIS_Hectares": 1594.5452365353703,
        "Source_Datasets": "Comb_National_NIFC_Interagency_Fire_Perimeter_History (1)",
        "Listed_Fire_Types": "Wildfire (1)",
        "Listed_Fire_Names": "Big Quilcene River (1)",
        "Listed_Fire_Codes": "No code provided (1)",
        "Listed_Fire_IDs": "",
        "Listed_Fire_IRWIN_IDs": "",
        "Listed_Fire_Dates": "Listed Other Fire Date(s): 2006-11-02 - NIFC DATE_CUR field (1)",
        "Listed_Fire_Causes": "",
        "Listed_Fire_Cause_Class": "Undetermined (1)",
        "Listed_Rx_Reported_Acres": null,
        "Listed_Map_Digitize_Methods": "Other (1)",
        "Listed_Notes": "",
        "Proce

In [7]:
#
#    Every feature has a 'geometry' which specifies geo coordinates that make up each geographic thing
#    In the case of the wildfire data, most wildfires are bounded shapes, circles, squares, etc. This is
#    represented by shapes called 'rings' in GeoJSON.
#
# Get the geometry for the feature we pulled from the feature_list
gj_geometry = gj_feature['geometry']
# The largest shape (ring) is supposed to be item zero in the list of 'rings'
gj_bigest_ring = gj_geometry['rings'][0]

print(f"The largest ring of gj_feature['features'][{SLOT}]['rings'] consists of {len(gj_bigest_ring)} points.")

The largest ring of gj_feature['features'][0]['rings'] consists of 768 points.


### Computing distance

One issue in performing geodetic computation is that any (all) geographic coordinate systems are eventually translated to the surface of the earth - which is not flat. That means every computation of distance between two points is some kind of arc (not actually a straight line). Further the earth is not a true sphere, its a type of ellipsoid. That means the amount of curvature varies depending upon where you are on the surface and the direction - which changes the distance.

 [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html) can help. It has functions that will convert coordinate points between (almost) any two of the many different geographic coordinate systems. As well, Pyproj provides ways to compute distances between two points (mostly assuming the points are already in the same coordinate system).

The city I was given as part of this assignment is Philadelphia, PA. I will use the Pyproj library to compute the distance between Philadelphia and the location of the wildfire data. I found the Lat long for this city using Wikipedia.

### Calculating the distance between Philadelphia and the wildfire data

#### Converting ESRI:102008 to EPSG:4326

In [13]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

In [23]:
#
#   Convert one ring from the default to EPSG
#
#   There are two options here - depending upon whether you loaded data useing GeoJSON or the wildfire.Reader
#
#ring_in_epsg4326 = convert_ring_to_epsg4326(gj_bigest_ring)
#
ring_in_epsg4326 = convert_ring_to_epsg4326(wf_bigest_ring)
#
print(f"Ring consists of {len(ring_in_epsg4326)} points.")
#
#    If you want to print them out you can see what they look like converted.
#print(ring_in_epsg4326)
#for point in ring_in_epsg4326:
#    print(f"{point[0]},{point[1]}")


Ring consists of 768 points.


#### Computing distance between a place and a wildfire

For futher analysis, we need to compute the distance between a place and a wildfire. The basic problem is knowing how far away a fire is from some location (like a city). One issue is that fires are irregularly shaped so the actual answer to that is a bit dependent upon the exact shape and how you want to think about the notion of 'distance'. 

The first function finds the point on the perimiter with the shortest distance to the city (place) and returns the distance as well as the lat,lon of the perimeter point.

The second function calculates the average distance of all perimeter points to the city (place) and returns that average as the distance. This is not quite what the centroid would be, but it is probably fairly close.

These are two reasonable ways to think about possible distance to a fire. But both require computing distance to a whole set of points on the perimeter of a fire.

For now, I am using both if these. I make a decision later as to which one to use.


In [8]:
#
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point



#
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average




Below is a list of fields I think will be useful for me to extract from the data to do the analysis further.

In [9]:
req_fields = [
  'OBJECTID',
  'USGS_Assigned_ID',
  'Assigned_Fire_Type',
  'Fire_Year',
  'Fire_Polygon_Tier',
  'Fire_Attribute_Tiers',
  'GIS_Acres',
  'GIS_Hectares',
  'Source_Datasets',
  'Listed_Fire_Types',
  'Listed_Fire_Names',
  'Listed_Fire_Codes',
  'Listed_Fire_IDs',
  'Listed_Fire_IRWIN_IDs',
  'Listed_Fire_Dates',
  'Listed_Fire_Causes',
  'Listed_Fire_Cause_Class',
  'Listed_Rx_Reported_Acres',
  'Listed_Map_Digitize_Methods',
  'Wildfire_and_Rx_Flag',
  'Overlap_Within_1_or_2_Flag',
  'Circleness_Scale',
  'Circle_Flag',
  'Exclude_From_Summary_Rasters',
  'Shape_Length',
  'Shape_Area',
]

#### Calculating actual distance, formatting data and saving it to a file

In [ ]:
#    Get a city from our CITY_LOCATIONS constant as our starting position
#place = CITY_LOCATIONS["tomales"]
#place = CITY_LOCATIONS["medford"]
#place = CITY_LOCATIONS["encinitas"]
#place = CITY_LOCATIONS["redding"]
#place = CITY_LOCATIONS["loveland"]
place = {
    "city":"Philadelphia",
    "latlon": [39.9526, -75.1652]
}
dict_of_fires = []
counters = 0
for wf_feature in gj_data['features']:
    try:
    #print(f"{place['city']}")
        feature_dict = {key:wf_feature['attributes'][key] for key in req_fields}
        wf_year = wf_feature['attributes']['Fire_Year']
        wf_name = wf_feature['attributes']['Listed_Fire_Names'].split(',')[0]
        wf_size = wf_feature['attributes']['GIS_Acres']
        wf_type = wf_feature['attributes']['Assigned_Fire_Type']
        if 'rings' in wf_feature['geometry']:
            ring_data = wf_feature['geometry']['rings'][0]
        elif 'curveRings' in wf_feature['geometry']:
            ring_data = wf_feature['geometry']['curveRings'][0]
        else:
            raise Exception("HEY! No compatible geometry in this fire data!!!")
        #
        #     Compute using the shortest distance to any point on the perimeter
        #
        
        distance = shortest_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
        feature_dict['shortest_dist'] = distance
        # print(f"The closest distance of fire '{wf_name}' ({wf_size:1.2f} acres) from {wf_year} was {distance[0]:1.2f} miles to {place['city']}")
        # print(f"\tThe cloest perimiter point lat,lon {distance[1][0]},{distance[1][1]}")
        
        #
        #     Compute using the average distance to all points on the perimeter
        #
        distance = average_distance_from_place_to_fire_perimeter(place['latlon'],ring_data)
        feature_dict['avg_dist'] = distance
        #print(f"Fire '{wf_name}' ({wf_size:1.2f} acres) from {wf_year} was an average {distance:1.2f} miles to {place['city']}")
        # just get a location to print thats on the ring (perimeter)
        ring = convert_ring_to_epsg4326(ring_data)
        perimeter_start = ring[0]
        #print(f"\tOne perimiter point lat,lon {perimeter_start[0]},{perimeter_start[1]}")
        #print("Count" + str(counters)+" : Done for : "+str(feature_count['OBJECTID'])+" "+str(feature_dict['shortest_dist'])+ ": "+ str(feature_dict['avg_dist']))
        counters+=1
        dict_of_fires.append(feature_dict)
        #print(counters)
        if(counters%200 == 0):
            print(counters)
    except Exception as e: 
        print(f"Error in processing fire {wf_feature['attributes']['OBJECTID']}")

Below are a few of the records that error out when trying to process them. I will need to remove these records from the dataset before I can proceed with the analysis.

It is only 0.001% of the data that is errored out. So, I will remove these records from the dataset and proceed with the analysis.
- Error in processing fire 109605
- Error in processing fire 110224
- Error in processing fire 110639
- Error in processing fire 111431
- Error in processing fire 111897
- Error in processing fire 112410
- Error in processing fire 112415
- Error in processing fire 113411
- Error in processing fire 113665
- Error in processing fire 113738
- Error in processing fire 113766
- Error in processing fire 113805
- Error in processing fire 114309
- Error in processing fire 114322
- Error in processing fire 115629
- Error in processing fire 115974
- Error in processing fire 116235
- Error in processing fire 117086
- Error in processing fire 119582
- Error in processing fire 119617
- Error in processing fire 119751
- Error in processing fire 119982
- Error in processing fire 120212
- Error in processing fire 120678
- Error in processing fire 121010
- Error in processing fire 122264
- Error in processing fire 125745
- Error in processing fire 127492

Saving as dictionary

In [25]:
save_dict = dict()
save_dict["values"] = dict_of_fires

Writing to a json file ans saving. Will be retrieved later for further analysis.

In [26]:
#write save_dict to a file
with open('fire_distance_data.json', 'w') as json_file:
    json.dump(save_dict, json_file)
    print("File written successfully")

File written successfully
